In [126]:
import pandas as pd
from datetime import timedelta

In [127]:
cruises = pd.read_csv('data/01_cruises.csv')

cruises = cruises[['Dato','ETA','ETD','Navn','Passasjerer','Kai', 'Agent','Tonn', 'Lengde']]

# Filling the times for World Traveler
row_index = 16
column_name = 'ETA'
value_to_fill = '08:00'

cruises.loc[row_index, column_name] = cruises.loc[row_index, column_name] if pd.notna(cruises.loc[row_index, column_name]) else value_to_fill

row_index = 16
column_name = 'ETD'
value_to_fill = '17:00'

cruises.loc[row_index, column_name] = cruises.loc[row_index, column_name] if pd.notna(cruises.loc[row_index, column_name]) else value_to_fill

# Filling the times for Jewel of the Sea which stayed overnight
row_index = 47
column_name = 'ETD'
value_to_fill = '23:00'

cruises.loc[row_index, column_name] = cruises.loc[row_index, column_name] if pd.notna(cruises.loc[row_index, column_name]) else value_to_fill

row_index = 49
column_name = 'ETA'
value_to_fill = '00:00'

cruises.loc[row_index, column_name] = cruises.loc[row_index, column_name] if pd.notna(cruises.loc[row_index, column_name]) else value_to_fill

# zbavily jsme se Queen Victoria na indexu 15, ktera nedoplula.
cruises = cruises.drop(15, axis='index') 


# # Kontrola toho, ze uz nemame v cruises hodnoty s nevyplnenym casem
# lode = cruises[['Navn','Passasjerer','Dato','ETA','ETD']] #timhle si vyberu, ktere tabulky maji chybu a kuzu si to zkontrolovat v tabulce s rozpisem sluzeb, kdy lode opravdu dojely

# nula = lode[lode.isna().any(axis=1)]
# nula



### Úprava data z formátu "dd.mm.yyyy" do formátu "yyyy-mm-dd":

In [128]:
cruises[["Day", "Month", "Year"]] = cruises["Dato"].str.split(".", expand=True)
cruises["Date of arrival"] = cruises["Year"] + "-" + cruises["Month"]+ "-" + cruises["Day"]
cruises["Date of departure"] = cruises["Year"] + "-" + cruises["Month"]+ "-" + cruises["Day"]
cruises = cruises.drop(["Dato", "Day", "Month", "Year"], axis="columns")
cruises = cruises[["Date of arrival","Date of departure",'ETA','ETD','Navn','Passasjerer','Kai', 'Agent','Tonn', 'Lengde']]

## Toto jsem s dovolením přesunula z té části s tvým kódem, ať to CSVčko tvoříme až po všech těch úpravách
# cruises.to_csv('data/cruises.csv', index=False) 




Nové sloupečky ETA date/ETD date - slepený čas + datum příjezdu/odjezdu lodě

In [129]:
cruises["ETA date"] = cruises["Date of arrival"] + " " + cruises["ETA"] + ":00"
cruises["ETD date"] = cruises["Date of arrival"] + " " + cruises["ETD"] + ":00"

# # Změní čas na typ datetime místo objectu, co to byl původně 
cruises["ETA date"] = pd.to_datetime(cruises["ETA date"], errors='coerce')
cruises["ETD date"] = pd.to_datetime(cruises["ETD date"], errors='coerce')

# # Vypočítá rozdíl v časech - vyhazuje to - 1 den když loď přijede před půlnocí a odjede na druhý den
cruises["Time in harbour"] = cruises["ETD date"] - cruises["ETA date"]

#index name, ať se mi na to líp hledí
cruises.index.name = "ID"


### Převedení sloupečků 'Date of arrival' a 'Date of departure' na typ datetime64[ns]

In [130]:
cruises['Date of arrival'] = pd.to_datetime(cruises['Date of arrival'])
cruises['Date of departure'] = pd.to_datetime(cruises['Date of departure'])


### Projdi tabulku cruises a pokud je ve sloupci "Time in harbour" rozdíl méně než 0 dní 0:00:00, přičti ke sloupečku "Date of departure" jeden den: 

In [131]:
# Definice podmínky, za které se přičte den navíc do sloupečku "Date of departure"
podminka_pridani_dne = cruises["Time in harbour"] < "0 days 00:00:00"

# Aplikace podmínky
cruises.loc[podminka_pridani_dne, "Date of departure"] = cruises["Date of departure"] + timedelta(days=1)

# zbavím se sloupečků, co nechci:
cruises = cruises.drop(["ETA date", "ETD date", "Time in harbour"], axis="columns")
cruises.head(20)
### Výsledná tabulka má date of departure o den větší než date of arrival, pokus je mezi časy příjezdu a odjezdu půlnoc


,Date of arrival,Date of departure,ETA,ETD,Navn,Passasjerer,Kai,Agent,Tonn,Lengde
ID,,,,,,,,,,
0,2023-01-13,2023-01-13,10:00,16:00,Otto Sverdrup,400.0,Berth Nord,HRX,15690,138
1,2023-01-27,2023-01-27,10:00,18:00,Maud,400.0,Berth Nord,Jacob Falck ANS,16151,136
2,2023-01-27,2023-01-27,10:00,16:00,Otto Sverdrup,400.0,Berth 3,HRX,15690,138
3,2023-02-10,2023-02-10,10:00,16:00,Otto Sverdrup,400.0,Berth Nord,HRX,15690,138
4,2023-02-24,2023-02-24,10:00,16:00,Otto Sverdrup,400.0,Berth Nord,HRX,15690,138
5,2023-02-25,2023-02-25,00:13,17:00,Spirit of Discovery,990.0,Berth North,Gac Norway AS,58119,237
6,2023-02-26,2023-02-26,00:15,23:30,Mein Schiff 3,2750.0,Berth Nord,EuropeanCruiseService,99526,294
7,2023-03-10,2023-03-10,10:00,16:00,Otto Sverdrup,400.0,Berth 3,HRX,15690,138
8,2023-03-10,2023-03-10,00:13,17:00,Spirit of Adventure,1000.0,Berth Nord,Gac Norway AS,58119,237


## Export do csvčka

In [132]:
cruises.to_csv('data/cruises.csv', index=False) 